In [1]:
# Autoimport changes in code
%load_ext autoreload
%autoreload 2

In [5]:
import sys, os
sys.path.append(os.path.dirname(os.getcwd())) # Include ../SSD in path
import numpy as np
import torch
import matplotlib.pyplot as plt
from vizer.draw import draw_boxes
from tops.config import instantiate, LazyConfig
from ssd import utils
np.random.seed(0)

In [6]:
# Only necessary to run this on local machine
# ['/data', '/dataset_exploration', '/datasets', '/demo', '']
%cd ../

/Users/mariu/dev/school/TDT4265/project/computer-vision-and-deep-learning/SSD


In [55]:
# Select run
run = "task2_1"
input_size = (3, 128, 1024)

In [56]:
from ssd.utils import load_config
cfg = load_config(f"configs/{run}.py")

Saving SSD outputs to: outputs/


In [57]:
from tops.config import instantiate
model = instantiate(cfg.model)

In [54]:
from torchsummary import summary

summary(model, input_size=input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1        [-1, 32, 128, 1024]             896
       BatchNorm2d-2        [-1, 32, 128, 1024]              64
         LeakyReLU-3        [-1, 32, 128, 1024]               0
            Conv2d-4        [-1, 64, 128, 1024]          18,496
       BatchNorm2d-5        [-1, 64, 128, 1024]             128
         LeakyReLU-6        [-1, 64, 128, 1024]               0
         MaxPool2d-7          [-1, 64, 64, 512]               0
            Conv2d-8         [-1, 128, 64, 512]          73,856
       BatchNorm2d-9         [-1, 128, 64, 512]             256
        LeakyReLU-10         [-1, 128, 64, 512]               0
           Conv2d-11         [-1, 128, 64, 512]         147,584
      BatchNorm2d-12         [-1, 128, 64, 512]             256
        LeakyReLU-13         [-1, 128, 64, 512]               0
        MaxPool2d-14         [-1, 128, 

In [29]:
dataset_to_visualize = "train" # or "val"

cfg.train.batch_size = 32

if dataset_to_visualize == "train":
    # Remove GroundTruthBoxesToAnchors transform
    if cfg.data_train.dataset._target_ == torch.utils.data.ConcatDataset:
        for dataset in cfg.data_train.dataset.datasets:
            print(dataset.transform.transforms)
            dataset.transform.transforms = dataset.transform.transforms[:-1]
    else:
        cfg.data_train.dataset.transform.transforms = cfg.data_train.dataset.transform.transforms[:-1]
    dataset = instantiate(cfg.data_train.dataloader)
    gpu_transform = instantiate(cfg.data_train.gpu_transform)
else:
    cfg.data_val.dataloader.collate_fn = utils.batch_collate
    dataset = instantiate(cfg.data_val.dataloader) 
    gpu_transform = instantiate(cfg.data_val.gpu_transform)

# Assumes that the first GPU transform is Normalize
# If it fails, just change the index from 0.
image_mean = torch.tensor(cfg.data_train.gpu_transform.transforms[0].mean).view(1, 3, 1, 1)
image_std = torch.tensor(cfg.data_train.gpu_transform.transforms[0].std).view(1, 3, 1, 1)

In [37]:
# dataiter = iter(dataset)
# sample = next(dataiter)

In [58]:
from torch.utils.tensorboard import SummaryWriter

# default `log_dir` is "runs" - we'll be more specific here
writer = SummaryWriter(f'runs/{run}')

In [62]:
dummy_input = torch.rand(1, 3, 128, 1024)
writer.add_graph(model, dummy_input)
writer.close()

/Users/mariu/dev/school/TDT4265/project/computer-vision-and-deep-learning/SSD/ssd/modeling/backbones/basic.py:96: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert feature.shape[1:] == expected_shape, \
/Users/mariu/dev/school/TDT4265/project/computer-vision-and-deep-learning/SSD/ssd/utils/box_utils.py:23: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert boxes_center.shape[-1] == 4


Only tensors, lists, tuples of tensors, or dictionary of tensors can be output from traced functions
Error occurs, No graph saved


RuntimeError: Only tensors, lists, tuples of tensors, or dictionary of tensors can be output from traced functions

In [33]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C
